In [18]:
import pandas as pd

In [19]:
df_esg = pd.read_csv('real_tabulation_scraped_data.csv', sep='\t')
df_roe = pd.read_csv('roe_results_tab.csv', sep='\t')
df_roe = df_roe.iloc[29:]
df_merge = pd.merge(df_esg, df_roe, on='name', how='inner')
df_merge['Year'] = df_merge["date_x"].str.extract(r'(\d{4})')
df_merge.drop(['date_x', 'date_y'], axis=1, inplace=True)
df_merge = df_merge.dropna(subset=['roe'])
df_merge

,name,grade,country,industry,employees,roe,Year
0,"1-800-FLOWERS.COM, Inc.",22.6,United States of America,Retailing,"4,000",-2.07%,2024
1,1&1 AG,27.7,Germany,Telecommunication Services,"3,280",4.34%,2024
2,"10X Genomics, Inc.",22.5,United States of America,Pharmaceuticals,"1,259",-24.87%,2024
3,11 Bit Studios SA,16.3,Poland,Software & Services,299,0.23%,2023
4,1st Source Corp.,35.0,United States of America,Banks,"1,170",12.01%,2024
...,...,...,...,...,...,...,...
7069,"The Hyakugo Bank, Ltd.",27.5,Japan,Banks,"2,792",3.43%,2024
7070,"The Hyakujushi Bank, Ltd.",29.0,Japan,Banks,"2,147",3.24%,2023
7073,The Italian Sea Group SpA,30.2,Italy,Machinery,710,30.74%,2023
7074,The J. M. Smucker Co.,27.2,United States of America,Food Products,"9,000",-1.18%,2023


In [20]:
df_merge['roe_num'] = df_merge['roe'].str.replace('%', '')
df_merge['roe_num'] = pd.to_numeric(df_merge['roe_num'], errors='coerce')
df_merge.to_csv('merged_data.csv', sep='\t', index=False)
df_merge = df_merge[df_merge['roe_num'] > -100]
df_merge = df_merge[df_merge['roe_num'] < 100] 

In [21]:
df_merge.sort_values(by='roe_num', ascending=False, inplace=True)
df_merge['positive'] = df_merge['roe_num'].apply(lambda x: 1 if x > 0 else 0)
df_merge

,name,grade,country,industry,employees,roe,Year,roe_num,positive
6903,"Tempur Sealy International, Inc.",12.7,United States of America,Consumer Durables,"13,500",98.08%,2024,98.08,1
3471,Gulfport Energy Corp.,43.0,United States of America,Oil & Gas Producers,226,95.30%,2023,95.30,1
3806,Leopalace21 Corp.,24.6,Japan,Real Estate,"3,853",94.11%,2023,94.11,1
6182,Softchoice Corp.,22.2,Canada,Software & Services,"2,100",93.93%,2023,93.93,1
26,4imprint Group Plc,16.5,United Kingdom,Media,"1,644",92.73%,2024,92.73,1
...,...,...,...,...,...,...,...,...,...
193,ADS-TEC Energy Plc,28.5,Ireland,Electrical Equipment,239,-96.07%,2023,-96.07,0
2747,"Foresee Pharmaceuticals Co., Ltd.",36.3,Taiwan,Pharmaceuticals,15,-96.17%,2023,-96.17,0
78,Abliva AB,29.2,Sweden,Pharmaceuticals,6,-96.71%,2024,-96.71,0
3905,Lions Gate Entertainment Corp.,18.1,United States of America,Media,"1,717",-97.66%,2023,-97.66,0


In [27]:
# Add a column that equal 'Europe' if the country is in Europe
european_country = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom']
df_merge['Europe'] = df_merge['country'].apply(lambda x: 'Europe' if x in european_country else 'Other')
df_europe = df_merge[df_merge['Europe'] == 'Europe']

usa_country = ['United States of America']
df_usa = df_merge[df_merge['country'].isin(usa_country)]


In [23]:
def select_datas(data, year, sector):
    df = data[(data['Year'] == year) & (data['industry'] == sector)]
    return df

def scale(datas):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(datas)
    return df_scaled

def elbow(datas):
    inertia = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, random_state=0)
        kmeans.fit(datas)
        inertia.append(kmeans.inertia_)
    fig = px.line(x=range(1, 11), y=inertia)
    fig.show()
    return inertia

In [29]:
for sector in df_usa['industry'].unique():
    df = select_datas(df_usa, '2023', sector)
    print(f"Secteur : {sector} : Taille du dataset : {df.shape[0]}")
# df = select_datas(df_europe, '2024', 'Software & Services')
# df_scaled = scale(df[['grade', 'roe_num']])
# elbow(df_scaled)

Secteur : Consumer Durables : Taille du dataset : 3
Secteur : Oil & Gas Producers : Taille du dataset : 6
Secteur : Software & Services : Taille du dataset : 57
Secteur : Aerospace & Defense : Taille du dataset : 5
Secteur : Building Products : Taille du dataset : 4
Secteur : Textiles & Apparel : Taille du dataset : 5
Secteur : Retailing : Taille du dataset : 21
Secteur : Consumer Services : Taille du dataset : 14
Secteur : Real Estate : Taille du dataset : 33
Secteur : Containers & Packaging : Taille du dataset : 3
Secteur : Media : Taille du dataset : 11
Secteur : Pharmaceuticals : Taille du dataset : 51
Secteur : Diversified Financials : Taille du dataset : 24
Secteur : Household Products : Taille du dataset : 5
Secteur : Machinery : Taille du dataset : 12
Secteur : Healthcare : Taille du dataset : 30
Secteur : Semiconductors : Taille du dataset : 9
Secteur : Utilities : Taille du dataset : 13
Secteur : Food Products : Taille du dataset : 10
Secteur : Traders & Distributors : Taille

In [25]:

df_merge_2024 = df_merge[df_merge['Year'] == '2024']
fig = px.scatter(
    df_merge_2024,
    x='grade',
    y='roe_num',
    color='roe_num',  # 'roe' utilisé pour la couleur continue
    hover_name='grade',
    color_continuous_scale=px.colors.sequential.Viridis,  # Exemple d'échelle de couleurs continue
    title='ESG Grade vs ROE in 2024'
)

# Afficher le graphique
fig.show()

# Plus la note est basse, mieux c'est

NameError: name 'px' is not defined